In [1]:
# Import libraries
"""
import numpy as np
import pandas as pd
import scipy as sp
import matplotlib.pyplot as plt
import seaborn as sns
from patsy import dmatrix
import datetime
"""

# Pretty display for notebooks
%matplotlib inline

# Ignore the warnings
import warnings
warnings.filterwarnings('ignore')

# Load the dataset
weather = pd.read_csv("../asset/weather.csv")

# Success - Display the first record
print("weather  data : ", weather.shape)

import awesome_functions as cf

# 원본을 유지하기 위해서 카피
df_weather = weather.copy()

# heatmap을 위한 cmap은 이거 계속 쓰시면 됩니다.
cmap = sns.light_palette("orange", as_cmap=True)

weather  data :  (20517, 20)


In [35]:
import pickle

In [83]:
df = pd.read_csv("df_train_with_station_nbr_201806202007.csv")

In [ ]:
item_9 = df

In [84]:
df.tail()

,date,store_nbr,item_nbr,units,station_nbr
4617595,2014-10-31,45,107,0,16
4617596,2014-10-31,45,108,0,16
4617597,2014-10-31,45,109,0,16
4617598,2014-10-31,45,110,0,16
4617599,2014-10-31,45,111,0,16


In [9]:
station_3 = cf.getSpecifiedWeatherData(df_weather, 3)

In [11]:
# station_1_t = cf.remove_m_row(station_1, ["tmax", "tmin", "tavg"])
print("")
print("")
station_3_dropped_all = cf.remove_m_row(station_3)



원본 데이터프레임 length : 1035
제거하고 싶은 컬럼 : ['station_nbr', 'date', 'tmax', 'tmin', 'tavg', 'depart', 'dewpoint', 'wetbulb', 'heat', 'cool', 'sunrise', 'sunset', 'codesum', 'snowfall', 'preciptotal', 'stnpressure', 'sealevel', 'resultspeed', 'resultdir', 'avgspeed']
제거한 int 컬럼 : ['tmax', 'tmin', 'tavg', 'dewpoint', 'wetbulb', 'heat', 'cool', 'sunrise', 'sunset']
제거한 float 컬럼 : ['depart', 'snowfall', 'preciptotal', 'stnpressure', 'sealevel', 'resultspeed', 'resultdir', 'avgspeed']
제거한 후 데이터프레임 length : 990


In [22]:
cf.remove_columns(station_1_dropped_all, ["date", "codesum", "station_nbr"])

제거한 컬럼명 : ['date', 'codesum', 'station_nbr']


,tmax,tmin,tavg,dewpoint,wetbulb,heat,cool,preciptotal,stnpressure,sealevel,resultspeed,resultdir,avgspeed
1030,61.0,41.0,51.0,38.0,44.0,14.0,0.0,0.00,29.70,29.89,6.3,27.0,7.3
1031,65.0,37.0,51.0,45.0,48.0,14.0,0.0,0.00,29.86,30.02,1.4,9.0,3.6
1032,66.0,48.0,57.0,51.0,53.0,8.0,0.0,0.11,29.70,29.86,2.9,27.0,4.4
1033,57.0,39.0,48.0,39.0,43.0,17.0,0.0,0.00,29.78,29.95,2.1,28.0,3.7
1034,53.0,40.0,47.0,36.0,41.0,18.0,0.0,0.00,29.87,30.03,5.4,4.0,5.8


In [29]:
def getFormula(df, categorical_columns, target_column):
    """
        df : 회귀분석할 df
        categorical_columns : 카테고리 컬럼명 리스트
        target_column : target column명 문자열
    """
    columns = df.columns
    formula = ""
#     formula = target_column + " ~ "
    for i, column in enumerate(columns):
        if column != target_column:
            if i != 0:
                formula += " + "
            if column in categorical_columns:
                formula += "C(" + column + ")"
            else:
                formula += "scale(" + str(column) + ")"
    return formula

In [32]:
from patsy import dmatrix

formula = getFormula(station_1_dropped_all, [], "")
station_1_dropped_all_ = dmatrix(formula, station_1_dropped_all, return_type="dataframe")

In [24]:
from statsmodels.stats.outliers_influence import variance_inflation_factor

vif = pd.DataFrame()
vif["VIF Factor"] = [variance_inflation_factor(station_1_dropped_all_.values, i) for i in range(station_1_dropped_all_.shape[1])]
vif["features"] = station_1_dropped_all_.columns
vif = vif.sort_values("VIF Factor").reset_index(drop=True)
vif

,VIF Factor,features
0,1.447042,preciptotal
1,1.528504,resultdir
2,14.907789,resultspeed
3,16.415881,avgspeed
4,92.648621,dewpoint
5,101.226962,sealevel
6,102.757359,stnpressure
7,176.840528,cool
8,332.320483,wetbulb
9,1248.429987,tmin


In [ ]:
from statsmodels.stats.outliers_influence import variance_inflation_factor

In [48]:
cols = ["tmin", "tmax", "heat", "wetbulb", "cool", "stnpressure", "sealevel", "dewpoint", "avgspeed", "resultspeed", "resultdir", "preciptotal"]

In [53]:
vif = pd.DataFrame()
vif["VIF Factor"] = [variance_inflation_factor(station_1_dropped_all[cols].values, i) for i in range(station_1_dropped_all[cols].shape[1])]
vif["features"] = station_1_dropped_all[cols].columns
vif = vif.sort_values("VIF Factor").reset_index(drop=True)
vif

,VIF Factor,features
0,1.627003e+00,preciptotal
1,7.969854e+00,resultdir
2,5.105479e+01,resultspeed
3,7.997444e+01,avgspeed
4,1.230679e+02,cool
5,4.605967e+02,dewpoint
6,2.195812e+03,heat
7,2.706414e+03,tmin
8,2.936064e+03,wetbulb
9,5.568547e+03,tmax


In [37]:
cols = ["dewpoint", "wetbulb"]
vif = pd.DataFrame()
vif["VIF Factor"] = [variance_inflation_factor(station_1_dropped_all[cols].values, i) for i in range(station_1_dropped_all[cols].shape[1])]
vif["features"] = station_1_dropped_all[cols].columns
vif = vif.sort_values("VIF Factor").reset_index(drop=True)
vif

,VIF Factor,features
0,51.653939,dewpoint
1,51.653939,wetbulb


In [33]:
vif = pd.DataFrame()
vif["VIF Factor"] = [variance_inflation_factor(station_1_dropped_all_.values, i) for i in range(station_1_dropped_all_.shape[1])]
vif["features"] = station_1_dropped_all_.columns
vif = vif.sort_values("VIF Factor").reset_index(drop=True)
vif

,VIF Factor,features
0,1.000000,Intercept
1,1.447042,scale(preciptotal)
2,1.528504,scale(resultdir)
3,14.907789,scale(resultspeed)
4,16.415881,scale(avgspeed)
5,92.648621,scale(dewpoint)
6,101.226962,scale(sealevel)
7,102.757359,scale(stnpressure)
8,332.320483,scale(wetbulb)
9,1248.429987,scale(tmin)
